# Introduction

After first attempts, the score was very low. (about 0.5)

Then, we lokked up other Kaggler's submissions, in particular Beatriz Justino's https://www.kaggle.com/code/beatrizjustino/lstm-disaster-tweets. (Version 12)

And we wondered, why it was so much better (about 0.8). Here are the findngs:

# Imports

In [1]:
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf
import numpy as np
import pandas as pd

# Imports that Justino's code needs:
from keras import preprocessing
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Embedding, LSTM, Dense, Dropout,BatchNormalization
from keras.models import Sequential
import tensorflow as tf
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

2024-08-13 13:01:28.014829: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-13 13:01:28.014921: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 13:01:28.020307: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Choices

In [2]:
BATCH_SIZE = 32
MAX_TOKENS = 10000
EPOCHS = 20
MAX_SENTENCE_LENGTH = 60

# Raw Datasets

In [3]:
all_known_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

train_df, val_df = train_test_split(all_known_df, test_size=0.2, random_state=42)

## Note that the data seems to be badly classified:
Two examples classified as disaster, which I stumbled about:

In [4]:
badly_classified_example = all_known_df[all_known_df["text"].str.contains("whirlwind of time")]
print(badly_classified_example)
print(badly_classified_example['text'].to_numpy())
print(badly_classified_example['target'].to_numpy())

badly_classified_example = all_known_df[all_known_df["text"].str.contains("Attack II Volleyball")]
print()
print(print(badly_classified_example))
print(badly_classified_example['text'].to_numpy())
print(badly_classified_example['target'].to_numpy())

         id    keyword                        location  \
7270  10409  whirlwind  Stamford & Cork (& Shropshire)   

                                                   text  target  
7270  I moved to England five years ago today. What ...       1  
['I moved to England five years ago today. What a whirlwind of time it has been! http://t.co/eaSlGeA1B7']
[1]

      id keyword location                                               text  \
466  674  attack      NaN  #volleyball Attack II Volleyball Training Mach...   

     target  
466       1  
None
['#volleyball Attack II Volleyball Training Machine - Sets Simulation - http://t.co/dCDeCFv934 http://t.co/dWBC1dUvdk']
[1]


# Justino's Raw Dataset

Justino encodes the target differently, and balances the uneven classes. (It does not seem to help a lot.)

In [5]:
def get_Justino_raw_data(df, df_val=None):
    def extract_text_and_target(df):
        return df["text"].to_numpy(), df["target"].to_numpy()
    class_0 = df[df['target'] == 0]
    class_1 = df[df['target'] == 1]
    class_0_under = class_0.sample(len(class_1), random_state=42)
    df_equalized = pd.concat([class_0_under, class_1])
    text, label = extract_text_and_target(df_equalized)
    if df_val is not None:
        text_val, label_val = extract_text_and_target(df_val)
        return text, label, text_val, label_val
    else:
        return text, label

# Our preprocessing functions

One function for cleaning text snippets:

In [6]:
import re, string
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english')
stop_words = set(stopwords.words('english'))

def clean_stem_text(text):    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove @mentions and hashtags
    text = re.sub(r'\@\w+|\#', '', text)
    # Remove digits
    text = re.sub(r'\d+', '', text)
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords and stem
    text = ' '.join([stemmer.stem(word) 
                     for word in text.split() 
                     if word not in stop_words])
    return text

def clean_stem_byte_tensor(tensor):
    texts = tensor.numpy()
    texts = np.reshape(texts, (-1,))
    texts = [text.decode("utf-8") for text in texts]
    texts = [clean_stem_text(text) for text in texts]
    return tf.constant(texts, dtype=tf.string, shape=(len(texts),))

Lots of functions for applying the cleaning function and for encoding the text numerically:

(One can later chose from these different methods.)

In [7]:
int_text_vectorization = keras.layers.TextVectorization(
    output_mode="int",
    max_tokens=MAX_TOKENS,
    output_sequence_length=MAX_SENTENCE_LENGTH
)
int_text_vectorization_custom_clean_text = keras.layers.TextVectorization(
    output_mode="int",
    max_tokens=MAX_TOKENS,
    output_sequence_length=MAX_SENTENCE_LENGTH,
    standardize=clean_stem_byte_tensor
)
tf_idf_text_vectorization_custom_clean_text = keras.layers.TextVectorization(
    output_mode="tf_idf",
    max_tokens=MAX_TOKENS,
    standardize=clean_stem_byte_tensor
)
tf_idf_text_vectorization = keras.layers.TextVectorization(
    output_mode="tf_idf",
    max_tokens=MAX_TOKENS
)
tf_idf_text_vectorization_custom_clean_text_ngram_2 = keras.layers.TextVectorization(
    output_mode="tf_idf",
    max_tokens=MAX_TOKENS,
    standardize=clean_stem_byte_tensor,
    ngrams=(1,2)
)
int_text_vectorization_custom_clean_text_ngram_2 = keras.layers.TextVectorization(
    output_mode="int",
    max_tokens=MAX_TOKENS,
    standardize=clean_stem_byte_tensor,
    ngrams=(1,2)
)

def preprocess_df(df, df_val=None, mode="int", clean="standard", ngrams=(1,), df_test=None, max_tokens=MAX_TOKENS,
                 output_sequence_length=MAX_SENTENCE_LENGTH):
    assert mode in ("int", "tf_idf")
    assert clean in ("standard", "custom")
    assert ngrams in [(1,), (1,2)]
    
    if clean=="standard":
        clean_method = "lower_and_strip_punctuation"
    elif clean=="custom":
        clean_method = clean_stem_byte_tensor
    
    # Need to silence output_sequence_length for the TextVectorization-layer if not in "int"-mode:
    if mode!="int": output_sequence_length=None
    
    vectorizer = keras.layers.TextVectorization(
        output_mode=mode,
        max_tokens=max_tokens,
        output_sequence_length=output_sequence_length,
        standardize=clean_method
    )
        
    def df_to_X_y(df):
        X = df["text"].to_numpy()
        y = np.reshape(np.array(df["target"]), (-1,1))
        return X, y
    
    X, y = df_to_X_y(df)
    vectorizer.adapt(X)
    if df_test is not None:
        X_test = vectorizer(df_test["text"].to_numpy())
        if df_val is not None:
            X_val, y_val = df_to_X_y(df_val)
            return vectorizer(X), y, vectorizer(X_val), y_val, X_test
        else:
            return vectorizer(X), y, X_test
    else:
        if X_val is not None:
            X_val, y_val = df_to_X_y(df_val)
            return vectorizer(X), y, vectorizer(X_val), y_val
        else:
            return vectorizer(X), y

(The proprocessing is safely run on the dataset (not in the model) to save the GPU from being blocked.)

# Justino's preprocessing functions
ATTENTION: There seemed to be a bug in applying the stemming! It does not work with lists, but should be applied on single words.

In [8]:
def clean_Justino(text_array):
    def process_sentence(sentence):
        sentence = re.sub(r'http\S+|www\S+|https\S+', '', sentence, flags=re.MULTILINE)
        sentence = re.sub(r'#\S+', '', sentence)
        sentence = sentence.translate(str.maketrans('', '', string.punctuation))
        words = sentence.split()
        filtered_words = [word for word in words if word.lower() not in ENGLISH_STOP_WORDS]
        return ' '.join(filtered_words)
    return [process_sentence(sentence) for sentence in text_array]

def get_Justino_preprocessed_data(df, df_val=None, label_to_cat=True, vocab_length=None, length_long_sentence=MAX_SENTENCE_LENGTH):
    if df_val is not None:
        texts, labels, texts_val, labels_val = get_Justino_raw_data(df, df_val)
        texts_val = clean_Justino(texts_val)
        texts_val = [stemmer.stem(word) for word in texts_val]
    else:
        texts, labels = get_Justino_raw_data(df)
    texts = clean_Justino(texts)
    #print(texts[:5])
    texts = [stemmer.stem(word) for word in texts]
    #print("-----------")
    #print(texts[:5])
    
    word_tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_length)
    word_tokenizer.fit_on_texts(texts)
    
    #from nltk.tokenize import word_tokenize
    #longest_train = max(text, key=lambda sentence: len(word_tokenize(sentence)))
    #length_long_sentence = len(word_tokenize(longest_train))
    
    def process_data(texts, labels):
        sequences = word_tokenizer.texts_to_sequences(texts)
        padded_sequences = pad_sequences(sequences, maxlen=length_long_sentence)
        if label_to_cat:
            labels = keras.utils.to_categorical(labels)
        return padded_sequences, labels
    
    padded_sequences, labels = process_data(texts, labels)
    if df_val is not None:
        padded_sequences_val, labels_val = process_data(texts_val, labels_val)
        return padded_sequences, labels, padded_sequences_val, labels_val
    else:
        return padded_sequences, labels
#get_Justino_preprocessed_data(train_df, val_df)

# F1-Metric

Using metrics=["f1_score"] is not working properly in tensorflow up until now!! The problem is discussed on some GitHub-site...

We specify an f1-metric more manually here:

In [9]:
f1 = tf.keras.metrics.F1Score(
    average=None, threshold=0.5, name='f1', dtype=None
)

For training, one could also try to use an adapted "f1-loss":

In [10]:
def f1_loss(y_true, y_pred):
    
    tp = tf.sum(tf.cast(y_true*y_pred, 'float'), axis=0)
    tn = tf.sum(tf.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = tf.sum(tf.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = tf.sum(tf.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + tf.epsilon())
    r = tp / (tp + fn + tf.epsilon())

    f1 = 2*p*r / (p+r+tf.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - tf.mean(f1)


# Our Models

In [11]:

def get_model(max_tokens=MAX_TOKENS, max_sentence_length=MAX_SENTENCE_LENGTH,
              hidden_dim=16, embedding_dim=33, mode="int"):
    if mode=="int":
        inputs = keras.Input(shape=(max_sentence_length,), dtype="int64")
        x = keras.layers.Embedding(input_dim=max_tokens, output_dim=embedding_dim)(inputs)
        x = keras.layers.GlobalAveragePooling1D()(x)
        #x = keras.layers.Flatten()(x)
    elif mode=="tf_idf":
        inputs = keras.Input(shape=(max_tokens,), dtype="int64")
        x = inputs
    
    x = keras.layers.Dense(hidden_dim, activation="relu")(x)
    x = keras.layers.Dense(hidden_dim, activation="relu")(x)+x
    #x = keras.layers.Dense(hidden_dim, activation="relu")(x)
    
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.Dense(1)(x)
    
    model = keras.Model(inputs=inputs, outputs=x)
    model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
                 #loss=f1_loss,
                 optimizer="rmsprop",
                 metrics=["accuracy", f1],
                 #run_eagerly=True
                 )
    return model

# We should also try to do simple logistic regression, when we find the time for it:

#def get_LR_model(max_tokens):
#    inputs = keras.Input(shape=(MAX_SENTENCE_LENGTH,), dtype="int64")
#    x = tf.keras.layers.CategoryEncoding(
#        num_tokens=max_tokens, output_mode='multi_hot', sparse=False
#    )(inputs)
#    x = keras.layers.Dense(1)(x)
#    model = keras.Model(inputs=inputs, outputs=x)
#    model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
#                 #loss=f1_loss,
#                 optimizer="adam",
#                 metrics=["f1_score", "accuracy", "precision", "recall"],
#                 #run_eagerly=True
#                 )
#    return model

#def get_tf_idf_LR_model(max_tokens):
#    inputs = keras.Input(shape=(max_tokens,), dtype="int64")
#    x = keras.layers.Dense(1)(inputs)
#    model = keras.Model(inputs=inputs, outputs=x)
#    model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
#                 #loss=f1_loss,
#                 optimizer="adam",
#                 metrics=["f1_score", "accuracy", "precision", "recall"],
#                 #run_eagerly=True
#                 )
#    return model

In [12]:
# That's basically Justino's model for our own data-formatting.. Maybe we'll use it one day.

#def get_LSTM_model():
#    inputs = keras.Input((MAX_SENTENCE_LENGTH,), dtype=tf.int64)
#    x = keras.layers.Embedding(input_dim=MAX_TOKENS, output_dim=128)(inputs)
#    x = keras.layers.Dropout(0.5)(x)
#    x = keras.layers.LSTM(65)(x)
#    x = keras.layers.Dense(1, activation="sigmoid")(x)
#    model=keras.Model(inputs, x)
#    model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=False),
#                  optimizer="Adamax",
#                  metrics=["accuracy", "f1_score"],
#                 #run_eagerly=True
#                 )
#    model.summary()
#    return model



# Jutino's Model

In [13]:
def get_Justino_model(max_words=MAX_TOKENS, max_len=MAX_SENTENCE_LENGTH):
    model = Sequential()
    model.add(Embedding(max_words, 128))
    model.add(Dropout(0.5))
    model.add(LSTM(64))
    model.add(Dense(2, activation='sigmoid'))

    model.compile(loss='BinaryCrossentropy', optimizer='Adamax', metrics=['accuracy', f1])
    return model

# Function for saving predictions

In [14]:
def predict_and_save(model, X_test, filename):
    predictions_logits = model.predict(X_test)
    predictions = tf.math.sigmoid(predictions_logits).numpy()
    predictions = (predictions>0.5).astype(int)
    df = test_df[["id"]].copy()
    df["target"] = predictions
    df.to_csv(filename, index=False)

# Runs

Justino orginal:

In [15]:
X_Justino, y_Justino = get_Justino_preprocessed_data(all_known_df)
mod_Justino = get_Justino_model(max_words=np.max(np.max(X_Justino))+1)
mod_Justino.fit(X_Justino, y_Justino, epochs=EPOCHS, batch_size=64)

Epoch 1/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 14s 97ms/step - accuracy: 0.5710 - f1: 0.5607 - loss: 0.6883
Epoch 2/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 10s 92ms/step - accuracy: 0.6810 - f1: 0.6826 - loss: 0.6138
Epoch 3/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 9s 90ms/step - accuracy: 0.7812 - f1: 0.7794 - loss: 0.4753
Epoch 4/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 9s 89ms/step - accuracy: 0.8213 - f1: 0.8198 - loss: 0.3947
Epoch 5/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 9s 91ms/step - accuracy: 0.8495 - f1: 0.8494 - loss: 0.3569
Epoch 6/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 9s 87ms/step - accuracy: 0.8534 - f1: 0.8530 - loss: 0.3400
Epoch 7/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 9s 83ms/step - accuracy: 0.8777 - f1: 0.8775 - loss: 0.2968
Epoch 8/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 9s 83ms/step - accuracy: 0.8820 - f1: 0.8809 - loss: 0.2808
Epoch 9/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 9s 89ms/step - accuracy: 0.9075 - f1: 0.9063 - loss: 0.2371
Epoch 10/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 8s 79ms/step - accuracy: 0.9098 - f1: 0.9096 - loss: 0.23

Justino with splitting data into train and val:

In [16]:
X_Justino, y_Justino, X_Justino_val, y_Justino_val = get_Justino_preprocessed_data(train_df, val_df)
mod_Justino = get_Justino_model(max_words=np.max(np.max(X_Justino))+1)
mod_Justino.fit(X_Justino, y_Justino, validation_data=(X_Justino_val, y_Justino_val), epochs=EPOCHS, batch_size=64)

Epoch 1/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 10s 81ms/step - accuracy: 0.5416 - f1: 0.8482 - loss: 0.6908 - val_accuracy: 0.6848 - val_f1: 0.6239 - val_loss: 0.6583
Epoch 2/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 6s 72ms/step - accuracy: 0.6470 - f1: 0.6351 - loss: 0.6498 - val_accuracy: 0.7301 - val_f1: 0.7209 - val_loss: 0.5768
Epoch 3/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 7s 84ms/step - accuracy: 0.7402 - f1: 0.7396 - loss: 0.5419 - val_accuracy: 0.7682 - val_f1: 0.7517 - val_loss: 0.4922
Epoch 4/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 8s 92ms/step - accuracy: 0.8145 - f1: 0.8160 - loss: 0.4157 - val_accuracy: 0.7781 - val_f1: 0.7647 - val_loss: 0.4741
Epoch 5/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 7s 85ms/step - accuracy: 0.8465 - f1: 0.8476 - loss: 0.3566 - val_accuracy: 0.7781 - val_f1: 0.7683 - val_loss: 0.4791
Epoch 6/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 7s 85ms/step - accuracy: 0.8667 - f1: 0.8664 - loss: 0.3176 - val_accuracy: 0.7748 - val_f1: 0.7689 - val_loss: 0.4897
Epoch 7/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 6s 79ms/step - accu

### Own models:

Using standard cleaning and multi-hot-encodig:

In [17]:
X_train, y_train, X_val, y_val, X_test = preprocess_df(train_df, val_df, df_test=test_df, mode="int", clean="standard")
model = get_model(MAX_TOKENS)
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=64,
         callbacks=[keras.callbacks.ModelCheckpoint("own_model_int_standard.keras",save_best_only=True)])

Epoch 1/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5774 - f1: 0.4279 - loss: 0.6858 - val_accuracy: 0.5739 - val_f1: 0.0000e+00 - val_loss: 0.6859
Epoch 2/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5687 - f1: 0.0000e+00 - loss: 0.6842 - val_accuracy: 0.5739 - val_f1: 0.0000e+00 - val_loss: 0.6801
Epoch 3/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5600 - f1: 0.0000e+00 - loss: 0.6859 - val_accuracy: 0.5739 - val_f1: 0.0000e+00 - val_loss: 0.6781
Epoch 4/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5704 - f1: 0.0000e+00 - loss: 0.6801 - val_accuracy: 0.5739 - val_f1: 0.0000e+00 - val_loss: 0.6763
Epoch 5/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5719 - f1: 0.0000e+00 - loss: 0.6763 - val_accuracy: 0.5739 - val_f1: 0.0000e+00 - val_loss: 0.6711
Epoch 6/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5571 - f1: 0.0000e+00 - loss: 0.6721 - val_accuracy: 0.5739 - val_f1: 0.0000e+00 - val_loss: 0.6540
Epoch 7/20
96/96 ━━

Run all possible encodings with the get_model():

In [18]:
histories = {}
for mode in ("int", "tf_idf"):
    for clean in ("standard", "custom"):
        for ngrams in [(1,), (1,2)]:
            name = "__".join([mode, clean, str(ngrams)])
            print()
            print("------------- Starting " + name + " ---------------------")
            X_train, y_train, X_val, y_val, X_test = preprocess_df(train_df, val_df, df_test=test_df,
                                                                   mode=mode, clean=clean, ngrams=ngrams)
            model = get_model(MAX_TOKENS, mode=mode)
            #model.summary()
            histories[name] = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=42, batch_size=64,
                 callbacks=[keras.callbacks.ModelCheckpoint(name+".keras", save_best_only=True)])
            
            # Save predictions with best model:
            model = keras.models.load_model(name+".keras")
            predict_and_save(model, X_test, "predictions__" + name)



------------- Starting int__standard__(1,) ---------------------
Epoch 1/42
96/96 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.5775 - f1: 0.3099 - loss: 0.6921 - val_accuracy: 0.5739 - val_f1: 0.0000e+00 - val_loss: 0.6856
Epoch 2/42
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5665 - f1: 0.0000e+00 - loss: 0.6880 - val_accuracy: 0.5739 - val_f1: 0.0000e+00 - val_loss: 0.6830
Epoch 3/42
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5727 - f1: 0.0000e+00 - loss: 0.6841 - val_accuracy: 0.5739 - val_f1: 0.0000e+00 - val_loss: 0.6823
Epoch 4/42
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5576 - f1: 0.0000e+00 - loss: 0.6871 - val_accuracy: 0.5739 - val_f1: 0.0000e+00 - val_loss: 0.6822
Epoch 5/42
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5688 - f1: 0.0000e+00 - loss: 0.6855 - val_accuracy: 0.5739 - val_f1: 0.0000e+00 - val_loss: 0.6818
Epoch 6/42
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5788 - f1: 0.0000e+00 - loss: 0.6820 - val_accuracy

In [19]:
best_model_name = ""
global_best_val_score = 0

for name, history in histories.items():
    print(name)
    mode, cleaning, ngrams = name.split("__")
    print(f"Validation-f1-scores for {mode}-encoding, {cleaning}-cleaning, {ngrams}-ngrams:")
    vals_f1 = tf.stack(history.history["val_f1"])[:,0].numpy() # Don't know why it is saved so intricated
    print(vals_f1)
    max_val = np.max(vals_f1)
    if max_val > global_best_val_score:
        global_best_val_score = max_val
        best_model_name = name
    print("The maximal f1-score: ", np.max(vals_f1))
    print()
print("The best model in total is ", best_model_name)

int__standard__(1,)
Validation-f1-scores for int-encoding, standard-cleaning, (1,)-ngrams:
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.16315047 0.03915662 0.36699504 0.33164552 0.6674655
 0.54838705 0.38423643 0.42155686 0.50225216 0.6556419  0.57659566
 0.6888749  0.71038246 0.5090497  0.40727267 0.67502403 0.7518573
 0.7316636  0.6715658  0.5853658  0.7095115  0.6068821  0.73669267]
The maximal f1-score:  0.7518573

int__standard__(1, 2)
Validation-f1-scores for int-encoding, standard-cleaning, (1, 2)-ngrams:
[0.         0.         0.         0.         0.         0.
 0.         0.         0.1020408  0.5541401  0.6949602  0.10480348
 0.672279   0.44339615 0.7003609  0.731788   0.7346278  0.30429125
 0.50340134 0.6607669  0.7285843  0.6077002  0.5966386  0.5234899
 0.57020354 0.4815668  0.7568388  0.56652355 0.74874    0.64924616
 0.5386313

# Submit

In [20]:
df = pd.read_csv("predictions__" + best_model_name)
df.to_csv("submission.csv", index=False)
df.head()

,id,target
0,0,1
1,2,0
2,3,1
3,9,0
4,11,1


# Helper function for recunstructing Sentences

In [21]:
def reconstruct_sentence(int_vec, vectorizer):
    voc = np.array(vectorizer.get_vocabulary())
    #print(int_vecs)
    #print(type(int_vecs))
    sentence = (" ".join(voc[int_vec]))
    return sentence
#for i in range(...):
#    print(reconstruct_sentence(...), "  ", y_train[i])

# Conclusion

It seems, that our own original submission where heavily overfitted.

Using the ModelCheckpoint-Callback helps a lot! (What else do we have the validation data for?!)

Also, Justino's training starts to improve MUCH earlier, probably because of the ordering in the data, leading to some kind of "Curriculum learning". 
(See the code line with
df_equalized = pd.concat([class_0_under, class_1]))